# Data Management for module 1-2 project

The data contained in this project comes from an experiment conducted in 2015 adressing the question of whether a parasite infection ca affect fertility of a wild passerine bird. Data consist of:

 * Experimental infection: infected and uninfected individuals
 * Behavioral observations: Social hierarchies
 * Sperm video analyses: Various standard measures obtained from a CASA.

This script will first show how the casa data was initially obtained using a script that automatically launches ImageJ and introduces the right parameters to perform the desired video analyses on the ejaculate of each bird. From each analysis, a .txt file is created with the data of each spermatozoon detected by the CASA. Then, we will summarize the data contained in each file, and subsequently we will merge those data into a single table. Finally, those data will be matched to a table containing the experimental group of each bird, their social rank, and the time after infection at which the measure was done.

## Automatizing the video analysis in ImageJ with the CASA plug-in

To perform a automated sperm video analysis, first the right parameters has to be determined. In particular, the number of frames to consider a track, the search ratio or number of pixels to consider a single track between frames, the minimum and maximum pixels to consider a spermatozoon head, and the length of the video are important determinants of the quality of the analysis. From previous data, we had determined that for the recording conditions and camera quality used to generate the videos, the size of the spermatozoon heads after the threshold of the image is manipulated to remove the tail varies between 34 and 76 pixels (mean ± 2 SE). Additionally, from previous experiments we determined that the optimal video length is 1 second.

To determine the other parameters, we selected 5 videos at random and ran the various combinations of serach ratio and tracked frames yielding to the following results:

<img src="Sperm_video_optimization/motility_hasli_15_par.jpg" alt="Drawing" style="width: 330px;" align="left"/>
<img src="Sperm_video_optimization/vcl_hasli_15_par.jpg" alt="Drawing" style="width: 330px;"align="left"/>
<img src="Sperm_video_optimization/tracks_hasli_15_par.jpg" alt="Drawing" style="width: 330px;" align="left"/>

Using this information, the video analyses were done using a minimum of 16 tracked frames and a search ratio of 13 pixels. The scrip bellow defines a total of 21 parameters for the CASA plug-in as follow:

 * a: Minimum number of pixels
 * b: Maximum number of pixels
 * c: Minimum tracked frames
 * d: Maximum speed between frames (a.k.a. search ratio)
 * e: Minimum VSL for motile
 * f: Minimum VAP for motile
 * g: Minimum VCL for motile
 * h: Low VAP speed
 * i: Maximum percentage of path with zero VAP
 * j: Maximum percentage of path with low VAP
 * k: Low VAP speed 2
 * l: Low VCL speed
 * m: High WOB
 * n: High LIN
 * o: High WOB 2
 * p: High LIN 2
 * q: Frame rate
 * r: Microns per 1000 pixels
 * s: Print xy-coordinates of tracked sperm
 * t: Print motion characteristics for all tracked cells
 * u: Print median values for motion characteristics

To have full flexibility on the data yield by the CASA, we conducted a fully unrestricted analaysis where parameters e to l are set to zero. However, high wobble and linearity were restricted based on previous data to 200. Finally, based on the specifications of the microscope and the camera, the video was recorded at 25 frames per second with a total of 729 micros per 1000 pixels (For details see Rojas et al 2015).

*NOTE:* Be aware that ImageJ has to be installed in the computer where the analsys will be conducted, and additionally the Computer Assisted Sperm Analyzer (CASA) has to be manually installed to your ImageJ. To obtain the CASA plug-in, refer to the following site: https://imagej.nih.gov/ij/plugins/casa.html. Further information about the plug-in can be found in Wilson-Leedy and Ingermann (2006).

In [1]:
# RUN THIS CODE IN TERMINAL AND MAKE SURE THE DIRECTORY PATHS ARE CORRECT

Folder = getDirectory("Here_your_path"); // Refer here to the directory that contains your sperm videos.

images = getFileList(Folder); // The videos are fully decompressed, so they are treated as consecutive single images


for (i=0; i<images.length; i++) {

   inputPath = Folder + images[i];
 
run("AVI...", "select=inputPath first=51 last=76"); // omits the 50 first frames and goes until the frame 76, which makes a second of analysis
name = getTitle;
txt_name =replace(name, ".avi",".txt");
run("8-bit");
setAutoThreshold("Default dark");
//run("Threshold...");
setThreshold(0, 110); // Modify the image so that only the heads of the sperm cells are visible.
setThreshold(0, 110);
setOption("BlackBackground", false);
run("Convert to Mask", "method=Default background=Dark black"); // Inverts the background and now the sperm cells look like black dot on a white background
run("CASA ", "a,=34.0 b,=76.000000000 c,=16.000000000 d,=13.00000000 e,=0.000 f,=0.000000000 g,=0.000000000 h,=0.00000 i,=0.000000000 j,=0.000000000 k,=0.000000000 l,=0.000000000 m,=200.000000000 n,=200.000000000 o,=200.000000000 p,=200.000000000 q,=25.000000000 r,=729.000000000 s,=0 t,=1");
close();
close();
saveAs("Results", "/Target_Directory/"+txt_name);
run ("Close");

}


setBatchMode(false);


SyntaxError: invalid syntax (<ipython-input-1-6a6c46beb2ab>, line 1)

## Summarizing the data obtained from CASA for each video

Each file contains 7 columns corresponding to the VCL, VAP, VSL, LIN, WOB, PROG, and BCF of each sperm cell tracked. Each row of the file corresponds to a single spermatozoon, and the last row of the file summarizes the data. The mentioned mesurements correspond to:

<img src="Images/CASA.jpg" alt="Drawing" style="width: 450px;" align="center"/>
Figure 1. Description of the different measurements assesed by the CASA plug-in in ImageJ. Amplitude of lateral head displacement (AHL) corresponds to wobble (WOB). Linearity (LIN) is defined as the ratio between VSL and VAP, and progression (PROG) is the average distance that a sperm progressed across the average path. Image reproduced from Malo et al. (2005).




In [232]:
# The following code will manage the data obtained from the CASA plug-in. Given that each video results in a single .txt file, we create a for loop to summarize them individually and merge all the data in a single table

import pandas as pd
import numpy as np
import os

path = 'CASA_Output/' # Define the path where the data is
files = [] # Empty vector to store the files
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.txt' in file:
            files.append(os.path.join(r, file)) # Create a list with all the files in the folder that end in .txt

col_names = ["VCL", "VAP", "VSL", "LIN", "WOB", "PROG", "BCF","VCL15", "VAP15", "VSL15", "LIN15", "WOB15", "PROG15", "BCF15","VCL10", "VAP10", "VSL10", "LIN10", "WOB10", "PROG10", "BCF10","VCL5", "VAP5", "VSL5", "LIN5", "WOB5", "PROG5", "BCF5","MOTILITY", "TRACKS", "VIDEO"]
sum_sperm_quality = pd.DataFrame(columns = col_names) # Create empty table that will gather all the summarized data

# Now, we run a for loop to summarize each file, and subsequently append it to the main table.
for i in range(len(files)):
    
    column_names = ["VCL", "VAP", "VSL", "LIN", "WOB", "PROG", "BCF"] # Columns names for initial table
    
    # Read .txt file skipping last line that contains summary. Use column_names, avoid first lane that is created due to the extra lane on the last line.
    df = pd.read_csv(files[i], header = None, skipfooter=1, engine='python', sep = ' ', usecols = range(1,8), names = column_names)
    
    # Total number of sperm counted in the analysis
    total_sperm = df.count()[1]
    
    # Filter for sperm that is motile. This values are based on previous research. Low motility in a counting chamber is often due to drifting. Finally, table by speed, which later will be used to find the 15% 10% and 5% fastest sperm.
    df_alive = df[(df['VCL']>=20) & (df['VAP']>=15) & (df['VSL']>=10)].sort_values(by = ["VCL", "VAP", "VSL"], ascending = False)
    
    # Total number of motile sperm
    motile_sperm = df_alive.count()[1]
    
    # Take video name
    video = [files[i][12:-5]]
    
    # Calculate the proportion of motile sperm
    motile = [motile_sperm / total_sperm]
    
    # Join the motility and the video name in a dataframe
    motility = pd.DataFrame({'MOTILITY': motile, 'TRACKS': total_sperm,'VIDEO': video})
    
    # Get the mean measurements for all the motile sperm and store it in av_motile
    av_motile = pd.DataFrame(np.reshape(np.array(df_alive.mean()), (1,7)), columns = ["VCL", "VAP", "VSL", "LIN", "WOB", "PROG", "BCF"])
    
    # Get the mean measurements for the fastest 15% motile sperm and append it to av_motile in a single row
    av_motile = pd.concat([av_motile, pd.DataFrame(np.reshape(np.array(df_alive[:int(round(motile_sperm*0.15,0))].mean()), (1,7)), columns = ["VCL15", "VAP15", "VSL15", "LIN15", "WOB15", "PROG15", "BCF15"])], axis = 1)
    
    # Get the mean measurements for the fastest 10% motile sperm and append it to av_motile in a single row
    av_motile = pd.concat([av_motile, pd.DataFrame(np.reshape(np.array(df_alive[:int(round(motile_sperm*0.10,0))].mean()), (1,7)), columns = ["VCL10", "VAP10", "VSL10", "LIN10", "WOB10", "PROG10", "BCF10"])], axis = 1)
    
    # Get the mean measurements for the fastest 5% motile sperm and append it to av_motile in a single row
    av_motile = pd.concat([av_motile, pd.DataFrame(np.reshape(np.array(df_alive[:int(round(motile_sperm*0.05,0))].mean()), (1,7)), columns = ["VCL5", "VAP5", "VSL5", "LIN5", "WOB5", "PROG5", "BCF5"])], axis = 1)
    
    # Append to av_motile the proportion on motile sperm and the sperm video
    av_motile = pd.concat([av_motile, motility], axis = 1)
    
    # Append the row of summarized data to the main table, and ignore the original index of av_motile (e.g. else all the rows would be called 0)
    sum_sperm_quality = sum_sperm_quality.append(av_motile, ignore_index = True)

sum_sperm_quality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 31 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   VCL       321 non-null    float64
 1   VAP       321 non-null    float64
 2   VSL       321 non-null    float64
 3   LIN       321 non-null    float64
 4   WOB       321 non-null    float64
 5   PROG      321 non-null    float64
 6   BCF       321 non-null    float64
 7   VCL15     314 non-null    float64
 8   VAP15     314 non-null    float64
 9   VSL15     314 non-null    float64
 10  LIN15     314 non-null    float64
 11  WOB15     314 non-null    float64
 12  PROG15    314 non-null    float64
 13  BCF15     314 non-null    float64
 14  VCL10     308 non-null    float64
 15  VAP10     308 non-null    float64
 16  VSL10     308 non-null    float64
 17  LIN10     308 non-null    float64
 18  WOB10     308 non-null    float64
 19  PROG10    308 non-null    float64
 20  BCF10     308 non-null    float6

In [226]:
# Run this line to save the data frame generated above. Remove the # before running the line

# sum_sperm_quality.to_csv("data/summary_ejaculate_quality.txt", sep = " ", index = False, decimal = ".")

In [233]:
bird_data = pd.read_csv("data/bird_datasheet.txt", header = 0, sep = '\t')
bird_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   DATE              336 non-null    object 
 1   SAMPLING_GROUP    336 non-null    int64  
 2   EXP_DAY           336 non-null    int64  
 3   MANIPULATION_DAY  336 non-null    int64  
 4   TREATMENT         336 non-null    object 
 5   AVIARY            336 non-null    int64  
 6   ID                336 non-null    object 
 7   COLOR_RING        336 non-null    object 
 8   SAMPLING_TIME     336 non-null    int64  
 9   BODY_MASS         336 non-null    float64
 10  VIDEO             336 non-null    object 
 11  CLOACA_HIGH1      336 non-null    float64
 12  CLOACA_HIGH2      336 non-null    object 
 13  CLOACA_HIGH3      336 non-null    float64
 14  CLOACA_WIDTH1     336 non-null    float64
 15  CLOACA_WIDTH2     336 non-null    object 
 16  CLOACA_WIDTH3     336 non-null    float64
 1

In [236]:
full_df = bird_data.merge(sum_sperm_quality, on = 'VIDEO')
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320 entries, 0 to 319
Data columns (total 50 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   DATE              320 non-null    object 
 1   SAMPLING_GROUP    320 non-null    int64  
 2   EXP_DAY           320 non-null    int64  
 3   MANIPULATION_DAY  320 non-null    int64  
 4   TREATMENT         320 non-null    object 
 5   AVIARY            320 non-null    int64  
 6   ID                320 non-null    object 
 7   COLOR_RING        320 non-null    object 
 8   SAMPLING_TIME     320 non-null    int64  
 9   BODY_MASS         320 non-null    float64
 10  VIDEO             320 non-null    object 
 11  CLOACA_HIGH1      320 non-null    float64
 12  CLOACA_HIGH2      320 non-null    object 
 13  CLOACA_HIGH3      320 non-null    float64
 14  CLOACA_WIDTH1     320 non-null    float64
 15  CLOACA_WIDTH2     320 non-null    object 
 16  CLOACA_WIDTH3     320 non-null    float64
 1

,VCL,VAP,VSL,LIN,WOB,PROG,BCF


'CASA_Output/spvid_124-.txt'

In [76]:
sum_sperm_quality["VIDEO"][sum_sperm_quality["VAP15"] == 'nan']

325

In [110]:


float(list(df_alive[:int(round(motile_sperm*0.10,0))].mean()))

TypeError: float() argument must be a string or a number, not 'list'

In [143]:
df_alive.mean()

VCL     75.496189
VAP     62.251880
VSL     55.788698
LIN      0.894460
WOB      0.840069
PROG     7.465786
BCF     10.149972
dtype: float64

In [188]:
for i in range(len(files)):
    print(files[i])

CASA_Output/spvid_061-.txt
CASA_Output/spvid_124-.txt
CASA_Output/spvid_036-.txt
CASA_Output/spvid_173-.txt
CASA_Output/spvid_020-.txt
CASA_Output/spvid_165-.txt
CASA_Output/spvid_309-.txt
CASA_Output/spvid_077-.txt
CASA_Output/.Rhistory
CASA_Output/spvid_132-.txt
CASA_Output/spvid_098-.txt
CASA_Output/spvid_149-.txt
CASA_Output/spvid_325-.txt
CASA_Output/spvid_260-.txt
CASA_Output/spvid_237-.txt
CASA_Output/spvid_108-.txt
CASA_Output/spvid_221-.txt
CASA_Output/spvid_333-.txt
CASA_Output/spvid_299-.txt
CASA_Output/spvid_276-.txt
CASA_Output/spvid_201-.txt
CASA_Output/spvid_128-.txt
CASA_Output/spvid_082-.txt
CASA_Output/spvid_256-.txt
CASA_Output/spvid_313-.txt
CASA_Output/spvid_190-.txt
CASA_Output/spvid_240-.txt
CASA_Output/spvid_305-.txt
CASA_Output/spvid_186-.txt
CASA_Output/spvid_169-.txt
CASA_Output/spvid_094-.txt
CASA_Output/spvid_283-.txt
CASA_Output/spvid_112-.txt
CASA_Output/spvid_057-.txt
CASA_Output/spvid_104-.txt
CASA_Output/spvid_041-.txt
CASA_Output/spvid_295-.txt
CASA_O